In [33]:
pip install git+https://github.com/Fridaplatform/SofttekLLMSDK.git

  Cloning https://github.com/Fridaplatform/SofttekLLMSDK.git to c:\users\joses\appdata\local\temp\pip-req-build-rtp7z99s
  Resolved https://github.com/Fridaplatform/SofttekLLMSDK.git to commit 0f11c8ed07889b9e13a360509e81b4703e03bc0f
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/Fridaplatform/SofttekLLMSDK.git 'C:\Users\joses\AppData\Local\Temp\pip-req-build-rtp7z99s'


In [34]:
import os
from softtek_llm.chatbot import Chatbot
from softtek_llm.models import OpenAI
from softtek_llm.cache import Cache
from softtek_llm.vectorStores import PineconeVectorStore
from softtek_llm.embeddings import OpenAIEmbeddings
from softtek_llm.schemas import Filter
from dotenv import load_dotenv

In [35]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if OPENAI_API_KEY is None:
    raise ValueError("OPENAI_API_KEY not found in .env file")

OPENAI_API_BASE = os.getenv("OPENAI_API_BASE")
if OPENAI_API_BASE is None:
    raise ValueError("OPENAI_API_BASE not found in .env file")

OPENAI_EMBEDDINGS_MODEL_NAME = os.getenv("OPENAI_EMBEDDINGS_MODEL_NAME")
if OPENAI_EMBEDDINGS_MODEL_NAME is None:
    raise ValueError("OPENAI_EMBEDDINGS_MODEL_NAME not found in .env file")

OPENAI_CHAT_MODEL_NAME = os.getenv("OPENAI_CHAT_MODEL_NAME")
if OPENAI_CHAT_MODEL_NAME is None:
    raise ValueError("OPENAI_CHAT_MODEL_NAME not found in .env file")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
if PINECONE_API_KEY is None:
    raise ValueError("PINECONE_API_KEY not found in .env file")

PINECONE_ENVIRONMENT = os.getenv("PINECONE_ENVIRONMENT")
if PINECONE_ENVIRONMENT is None:
    raise ValueError("PINECONE_ENVIRONMENT not found in .env file")

PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
if PINECONE_INDEX_NAME is None:
    raise ValueError("PINECONE_INDEX_NAME not found in .env file")

In [36]:
vector_store = PineconeVectorStore(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENVIRONMENT,
    index_name=PINECONE_INDEX_NAME,
)
embeddings_model = OpenAIEmbeddings(
    api_key=OPENAI_API_KEY,
    model_name=OPENAI_EMBEDDINGS_MODEL_NAME,
    api_type="azure",
    api_base=OPENAI_API_BASE,
)
cache = Cache(
    vector_store=vector_store,
    embeddings_model=embeddings_model,
)
model = OpenAI(
    api_key=OPENAI_API_KEY,
    model_name=OPENAI_CHAT_MODEL_NAME,
    api_type="azure",
    api_base=OPENAI_API_BASE,
    verbose=True,
)
filters = [
    Filter(
        type="DENY",
        case="ANYTHING about the Titanic. YOU CANNOT talk about the Titanic AT ALL.",
    )
]
chatbot = Chatbot(
    model=model,
    description="You are a very helpful and polite chatbot",
    filters=filters,
    cache=cache,
    verbose=True,
)

In [37]:
response = chatbot.chat(
    "Hello! My name is Jeff.",
    print_cache_score=True,
    cache_kwargs={"namespace": "chatbot-test"},
)

Revision prompt: Considering this context:
	- user: Hello! My name is Jeff.
	- user: What is my name?

Please review the prompt below and answer with "yes" if it adheres to the rules or "no" if it violates any of the rules.
Rules:
	- DENY: ANYTHING about the Titanic. YOU CANNOT talk about the Titanic AT ALL.

Prompt:
Hello! My name is Jeff.
Memory: [{'role': 'system', 'content': "As a prompt reviser, your role is to determine whether the given prompt respects the rules provided below. You can only respond with 'yes' or 'no.' The rules are absolute, meaning that if any of them are not respected, the prompt is considered invalid."}, {'role': 'system', 'content': "only respond with 'yes' or 'no'"}, {'role': 'user', 'content': 'Considering this context:\n\t- user: Hello! My name is Jeff.\n\t- user: What is my name?\n\nPlease review the prompt below and answer with "yes" if it adheres to the rules or "no" if it violates any of the rules.\nRules:\n\t- DENY: ANYTHING about the Titanic. YOU CA

In [38]:
response

Response(message=Message(role='assistant', content='Hello Jeff! How can I assist you today?'), created=1695518687, latency=438, from_cache=False, model='gpt-35-turbo-16k', usage=Usage(prompt_tokens=47, completion_tokens=10, total_tokens=57), additional_kwargs={})

In [39]:
response = chatbot.chat(
    "What is my name?",
    print_cache_score=True,
    cache_kwargs={"namespace": "chatbot-test"},
)

Revision prompt: Considering this context:
	- user: What is my name?
	- user: Hello! My name is Jeff.
	- assistant: Hello Jeff! How can I assist you today?

Please review the prompt below and answer with "yes" if it adheres to the rules or "no" if it violates any of the rules.
Rules:
	- DENY: ANYTHING about the Titanic. YOU CANNOT talk about the Titanic AT ALL.

Prompt:
What is my name?
Memory: [{'role': 'system', 'content': "As a prompt reviser, your role is to determine whether the given prompt respects the rules provided below. You can only respond with 'yes' or 'no.' The rules are absolute, meaning that if any of them are not respected, the prompt is considered invalid."}, {'role': 'system', 'content': "only respond with 'yes' or 'no'"}, {'role': 'user', 'content': 'Considering this context:\n\t- user: What is my name?\n\t- user: Hello! My name is Jeff.\n\t- assistant: Hello Jeff! How can I assist you today?\n\nPlease review the prompt below and answer with "yes" if it adheres to t

MaxRetryError: HTTPSConnectionPool(host='default-unknown.svc.8c44f0d3-bc0e-49bf-b580-2c6bd6f2398f.pinecone.io', port=443): Max retries exceeded with url: /query (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001E79E5FA6D0>: Failed to resolve 'default-unknown.svc.8c44f0d3-bc0e-49bf-b580-2c6bd6f2398f.pinecone.io' ([Errno 11001] getaddrinfo failed)"))

In [ ]:
response

NameError: name 'response' is not defined

In [ ]:
response = chatbot.chat(
    "When did the Titanic sink?",
    print_cache_score=True,
    cache_kwargs={"namespace": "chatbot-test"},
)

Revision prompt: Considering this context:
	- user: Hello! My name is Jeff.
	- user: What is my name?

Please review the prompt below and answer with "yes" if it adheres to the rules or "no" if it violates any of the rules.
Rules:
	- DENY: ANYTHING about the Titanic. YOU CANNOT talk about the Titanic AT ALL.

Prompt:
When did the Titanic sink?
Memory: [{'role': 'system', 'content': "As a prompt reviser, your role is to determine whether the given prompt respects the rules provided below. You can only respond with 'yes' or 'no.' The rules are absolute, meaning that if any of them are not respected, the prompt is considered invalid."}, {'role': 'system', 'content': "only respond with 'yes' or 'no'"}, {'role': 'user', 'content': 'Considering this context:\n\t- user: Hello! My name is Jeff.\n\t- user: What is my name?\n\nPlease review the prompt below and answer with "yes" if it adheres to the rules or "no" if it violates any of the rules.\nRules:\n\t- DENY: ANYTHING about the Titanic. YOU

InvalidPrompt: The prompt does not follow the rules set by the filters. If this behavior is not intended, consider modifying the filters. It is recommended to use LLMs for meta prompts.

In [32]:
chatbot.cache.vector_store.delete(delete_all=True, namespace="chatbot-test")

MaxRetryError: HTTPSConnectionPool(host='default-unknown.svc.8c44f0d3-bc0e-49bf-b580-2c6bd6f2398f.pinecone.io', port=443): Max retries exceeded with url: /vectors/delete (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001E79E0E3610>: Failed to resolve 'default-unknown.svc.8c44f0d3-bc0e-49bf-b580-2c6bd6f2398f.pinecone.io' ([Errno 11001] getaddrinfo failed)"))